In [ ]:
from albumentations.core.composition import KeypointParams
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import numpy as np
from stacked_hourglass.predictor import HumanPosePredictor, GenericPosePredictor
from stacked_hourglass.model import hg2
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import csv
import glob
import random
import copy
from PIL import Image
import albumentations as A
from stacked_hourglass.utils import finetune
from stacked_hourglass import train as t
from stacked_hourglass.datasets.mpii import Mpii
from torch.nn import DataParallel
from stacked_hourglass.utils.logger import Logger
import os
from stacked_hourglass.utils.misc import save_checkpoint, adjust_learning_rate
from stacked_hourglass.train import do_training_epoch, do_validation_epoch
import tqdm
import argparse
from stacked_hourglass.datasets.mpii import MPII_JOINT_NAMES
import cv2
from torch.optim.rmsprop import RMSprop
from torch.utils.tensorboard import SummaryWriter
#polynomial regression separately
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=3)
X_poly = poly_reg.fit_transform(input1)
lin_reg2 = LinearRegression()
lin_reg2.fit(X_poly,output1)
x_poly_pred=lin_reg2.predict(poly_reg.fit_transform(input1))


poly_reg1 = PolynomialFeatures(degree=178)
X_poly1 = poly_reg1.fit_transform(input11)
lin_reg21 = LinearRegression()
lin_reg21.fit(X_poly1,output11)
x_poly_pred1=lin_reg21.predict(poly_reg1.fit_transform(input11))


#####################################
input111= df_x[['xx','yy','zz']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
output111=df_x[['output_x','output_y']]
poly_reg11 = PolynomialFeatures(degree=4)
X_poly11 = poly_reg11.fit_transform(input111)
lin_reg211 = LinearRegression()
lin_reg211.fit(X_poly11,output111)
x_poly_pred11=lin_reg211.predict(poly_reg11.fit_transform(input111))


#draw the real and predicted results
root="C:/Users/mobinamb/Desktop/Mobina work/pytorch pose estimation/transferlearning/pytorch-stacked-hourglass -571t/src/dataset/images/RGB/"
import os
image_id=['s01_e01/colorImg1448.jpg','s01_e01/colorImg1550.jpg','s01_e01/colorImg1388.jpg','s02_e01/colorImg782.jpg','s02_e01/colorImg660.jpg','s03_e02/colorImg76.jpg','s03_e02/colorImg116.jpg','s03_e02/colorImg622.jpg','s04_e02/colorImg904.jpg','s06_e01/colorImg1340.jpg']
j=100
for i in range(5,10):
    image=cv2.imread(root+image_id[i])
    k=0
    while k<20:
        cv2.circle(image,(int(x_poly_pred11[j][0]),int(x_poly_pred11[j][1])),radius=3,color=(0,255,0),thickness=-1)
        cv2.circle(image,(int(np.array(output1)[j][0]),int(np.array(output11)[j][0])),radius=5,color=(255,0,255),thickness=-1)
        k+=1
        j+=1
    break
plt.imshow(image)
#get 10 labels in pixel manually and find a correct formula by regression 
#run in IDLE
'''import matplotlib.pyplot as plt
import glob
import cv2

final_data_unchanged,final_data_converted={},{}
root="C:/Users/mobinamb/Desktop/Mobina work/pytorch pose estimation/transferlearning/pytorch-stacked-hourglass -571t/src/dataset/images/RGB/"
import os
image_id=['s01_e01/colorImg1448.jpg','s01_e01/colorImg1550.jpg','s01_e01/colorImg1388.jpg','s02_e01/colorImg782.jpg','s02_e01/colorImg660.jpg','s03_e02/colorImg76.jpg','s03_e02/colorImg116.jpg','s03_e02/colorImg622.jpg','s04_e02/colorImg904.jpg','s06_e01/colorImg1340.jpg']
pts=[]
for i in range(7,10):
    image=cv2.imread(root+image_id[i])
    plt.imshow(image)
    pts.append(plt.ginput(20,timeout=-1))

pts1=[[(199.2136359807968, 305.52139307672866), (199.2136359807968, 305.52139307672866), (190.58902552008763, 189.0891518571549), (195.97940705803083, 162.13724416743872), (216.46285690221512, 191.24530447233218), (137.76328644824395, 202.02606754821863), (256.35168028299506, 236.5245093910553), (134.529057525478, 264.5544933883601), (241.258611976754, 185.85492293438892), (150.7002021393077, 320.6144613829697), (229.39977259327893, 170.7618546281479), (147.46597321654176, 334.6294533816221), (230.47784890086754, 299.0529352311968), (161.48096521519417, 311.98985092226053), (225.0874673629243, 380.9867346079339), (189.51094921249896, 389.61134506864306), (205.68209382632867, 452.1397709087845), (166.87134675313737, 456.4520761391391), (236.94630674639942, 471.5451444453802), (190.58902552008763, 478.013602290912)], [(207.8116883116883, 306.51298701298697), (207.8116883116883, 306.51298701298697), (200.01948051948048, 183.13636363636357), (206.512987012987, 159.75974025974017), (237.68181818181816, 198.72077922077915), (151.96753246753244, 198.72077922077915), (235.08441558441555, 253.2662337662337), (164.9545454545454, 262.3571428571428), (251.9675324675325, 305.21428571428567), (158.46103896103895, 316.90259740259734), (253.26623376623374, 319.49999999999994), (158.46103896103895, 329.8896103896103), (240.27922077922076, 301.31818181818176), (168.85064935064932, 307.8116883116883), (227.2922077922078, 384.43506493506493), (189.6298701298701, 388.3311688311688), (205.2142857142857, 449.37012987012986), (164.9545454545454, 457.1623376623376), (236.38311688311686, 470.1493506493506), (187.0324675324675, 475.3441558441558)], [(296.12337662337666, 300.01948051948045), (296.12337662337666, 300.01948051948045), (298.72077922077926, 189.62987012987003), (294.82467532467535, 162.35714285714278), (329.8896103896104, 198.72077922077915), (251.9675324675325, 194.82467532467524), (337.68181818181824, 250.6688311688311), (242.87662337662337, 246.7727272727272), (331.1883116883117, 258.4610389610389), (305.21428571428567, 258.4610389610389), (325.9935064935065, 263.6558441558441), (319.5, 255.86363636363632), (324.6948051948052, 300.01948051948045), (258.461038961039, 306.51298701298697), (311.7077922077922, 385.73376623376623), (277.9415584415584, 396.12337662337654), (284.43506493506493, 454.564935064935), (225.9935064935065, 455.8636363636363), (319.5, 467.551948051948), (235.08441558441555, 475.3441558441558)], [(264.9545454545455, 327.2922077922077), (264.9545454545455, 327.2922077922077), (251.9675324675325, 241.57792207792204), (251.9675324675325, 216.90259740259734), (288.33116883116884, 246.7727272727272), (209.1103896103896, 250.6688311688311), (296.12337662337666, 294.82467532467524), (193.52597402597402, 298.72077922077915), (314.3051948051948, 310.4090909090909), (214.30519480519482, 314.3051948051948), (318.2012987012987, 315.6038961038961), (222.09740259740258, 328.590909090909), (288.33116883116884, 315.6038961038961), (223.39610389610388, 337.68181818181813), (318.2012987012987, 353.2662337662337), (254.56493506493504, 364.9545454545454), (297.42207792207796, 431.18831168831167), (235.08441558441555, 437.68181818181813), (324.6948051948052, 448.07142857142856), (242.87662337662337, 468.8506493506493)], [(327.2922077922078, 300.01948051948045), (328.5909090909091, 298.72077922077915), (325.9935064935065, 177.94155844155836), (307.8116883116883, 159.75974025974017), (359.7597402597403, 192.22727272727263), (318.2012987012987, 187.03246753246748), (371.448051948052, 259.75974025974017), (311.7077922077922, 261.0584415584415), (340.27922077922074, 319.49999999999994), (302.6168831168832, 290.9285714285714), (332.48701298701303, 335.0844155844155), (293.52597402597405, 298.72077922077915), (355.8636363636364, 314.3051948051948), (305.21428571428567, 302.61688311688306), (309.1103896103896, 380.538961038961), (297.42207792207796, 359.7597402597402), (345.47402597402595, 458.4610389610389), (388.33116883116884, 425.99350649350646), (300.01948051948057, 477.9415584415584), (351.96753246753246, 450.66883116883116)]]
pts2=[[(79.24025974025972, 272.7467532467532), (79.24025974025972, 272.7467532467532), (64.95454545454541, 149.3701298701298), (58.46103896103892, 122.09740259740249), (110.40909090909088, 164.95454545454538), (10.409090909090864, 164.95454545454538), (110.40909090909088, 238.98051948051943), (1.3181818181817704, 237.68181818181813), (124.69480519480518, 274.0454545454545), (44.175324675324646, 284.4350649350649), (116.90259740259737, 279.24025974025966), (57.16233766233762, 285.7337662337662), (106.512987012987, 288.3311688311688), (15.603896103896062, 292.22727272727263), (96.1233766233766, 376.6428571428571), (63.655844155844136, 384.43506493506493), (79.24025974025972, 450.66883116883116), (50.66883116883113, 463.65584415584414), (107.8116883116883, 461.05844155844153), (88.33116883116881, 477.9415584415584)], [(103.91558441558439, 284.4350649350649), (103.91558441558439, 284.4350649350649), (85.7337662337662, 149.3701298701298), (88.33116883116881, 123.3961038961038), (127.29220779220776, 168.8506493506493), (28.590909090909065, 170.1493506493506), (124.69480519480518, 237.68181818181813), (6.512987012986969, 241.57792207792204), (148.07142857142856, 264.9545454545454), (66.25324675324671, 284.4350649350649), (148.07142857142856, 284.4350649350649), (75.34415584415581, 302.61688311688306), (132.48701298701297, 284.4350649350649), (38.98051948051943, 298.72077922077915), (142.87662337662337, 389.6298701298701), (75.34415584415581, 398.72077922077915), (148.07142857142856, 477.9415584415584), (49.37012987012983, 467.551948051948), (197.42207792207793, 467.551948051948), (88.33116883116881, 476.6428571428571)]]
pts3=[[(438.98051948051943, 329.8896103896103), (438.98051948051943, 329.8896103896103), (438.98051948051943, 237.68181818181813), (433.78571428571433, 219.49999999999994), (477.9415584415584, 253.2662337662337), (400.01948051948057, 250.6688311688311), (496.12337662337666, 301.31818181818176), (389.62987012987014, 300.01948051948045), (476.6428571428571, 318.20129870129864), (410.4090909090909, 313.0064935064935), (472.7467532467532, 327.2922077922077), (414.3051948051948, 324.69480519480516), (467.5519480519481, 318.20129870129864), (409.1103896103896, 314.3051948051948), (497.42207792207796, 358.4610389610389), (385.73376623376623, 344.17532467532465), (457.1623376623377, 432.4870129870129), (418.2012987012987, 432.4870129870129), (489.62987012987014, 459.7597402597402), (387.03246753246754, 453.26623376623377)], [(136.38311688311686, 287.0324675324675), (136.38311688311686, 287.0324675324675), (127.29220779220776, 171.4480519480519), (148.07142857142856, 155.86363636363626), (170.14935064935062, 179.24025974025966), (77.94155844155841, 187.03246753246748), (170.14935064935062, 240.27922077922074), (84.4350649350649, 263.6558441558441), (181.83766233766235, 290.9285714285714), (103.91558441558439, 303.91558441558436), (180.53896103896105, 306.51298701298697), (111.70779220779218, 316.90259740259734), (175.34415584415584, 287.0324675324675), (103.91558441558439, 293.52597402597394), (159.75974025974025, 361.0584415584415), (97.4220779220779, 375.3441558441558), (129.88961038961037, 438.98051948051943), (111.70779220779218, 442.87662337662334), (161.05844155844156, 451.96753246753246), (77.94155844155841, 464.95454545454544)], [(368.8506493506494, 285.7337662337662), (368.8506493506494, 285.7337662337662), (362.3571428571429, 167.551948051948), (374.0454545454546, 138.98051948051938), (400.01948051948057, 177.94155844155836), (315.6038961038961, 177.94155844155836), (394.82467532467535, 249.3701298701298), (306.51298701298697, 255.86363636363632), (415.6038961038961, 287.0324675324675), (322.0974025974026, 307.8116883116883), (418.2012987012987, 310.4090909090909), (324.6948051948052, 323.39610389610385), (390.92857142857144, 288.3311688311688), (336.38311688311694, 290.9285714285714), (374.0454545454546, 359.7597402597402), (375.3441558441558, 379.2402597402597), (314.3051948051948, 403.91558441558436), (375.3441558441558, 475.3441558441558), (346.77272727272725, 436.3831168831168), (402.6168831168832, 477.9415584415584)]]
'''
import numpy as np
import pandas as pd 
#the real worl labels are in real label for regressoin.txt
with open('real label for regression.txt','r') as file:
    real=file.read()

#the pixel labels are in pts.npy
pixel=np.load('pts.npy')
output_x,output_y=[],[]
for i in range(len(pixel)):
    for j in range(len(pixel[i])):
        output_x.append(pixel[i][j][0])
        output_y.append(pixel[i][j][1])
#in the first regression model the inputs are the (x,z) of the points 
xx,yy,zz=[],[],[]
r=real.split()
for i in range(0,len(r),3):
    xx.append(float(r[i]))
    yy.append(float(r[i+1]))
    zz.append(float(r[i+2]))


pose = {'xx': xx,
        'yy': yy,
        'zz': yy,
        'output_x':output_x,
        'output_y':output_y
                }

df_x = pd.DataFrame(pose,columns=['xx','yy','zz','output_x','output_y'])
input1= df_x[['xx','zz']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
output1=df_x[['output_x']]

#regression model

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
model =LinearRegression()
model.fit(input1,output1) 

y_pred= model.predict(input1)
mean_squared_error(output1, y_pred, squared=False)

#in the second regression model the inputs are the (y,z) of the points 

input11= df_x[['yy','zz']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
output11=df_x[['output_y']]

#regression model

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
modell =LinearRegression()
modell.fit(input11,output11) 
y_pred1= modell.predict(input11)
mean_squared_error(output11, y_pred1, squared=False)

y_pred1

#draw the real and predicted results
root="C:/Users/mobinamb/Desktop/Mobina work/pytorch pose estimation/transferlearning/pytorch-stacked-hourglass -571t/src/dataset/images/RGB/"
import os
image_id=['s01_e01/colorImg1448.jpg','s01_e01/colorImg1550.jpg','s01_e01/colorImg1388.jpg','s02_e01/colorImg782.jpg','s02_e01/colorImg660.jpg','s03_e02/colorImg76.jpg','s03_e02/colorImg116.jpg','s03_e02/colorImg622.jpg','s04_e02/colorImg904.jpg','s06_e01/colorImg1340.jpg']
j=0
for i in range(10):
    image=cv2.imread(root+image_id[i])
    k=0
    while k<20:
        cv2.circle(image,(int(y_pred[j][0]),int(y_pred1[j][0])),radius=3,color=(255,0,0),thickness=-1)
        cv2.circle(image,(int(np.array(output1)[j][0]),int(np.array(output11)[j][0])),radius=5,color=(255,0,255),thickness=-1)
        k+=1
        j+=1
    break
plt.imshow(image)